In [1]:
import networkx as nx
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
from scipy.sparse import coo_matrix,csr_matrix

import timeit
%alias_magic t timeit

import sys
sys.path.insert(1, '../')

import SpringRank_tools as sr
import hunter as hw
import tools as tl
import cProfile

import pstats
from pstats import SortKey

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [2]:
def get_network(N = 50,p = 0.1):
    adjacency_list = []
    for i in range(N):
        for j in range(N):
            if np.random.rand() < p:
                adjacency_list.append([i,j])
    A = np.zeros([N,N])
    for edge in adjacency_list:
        A[edge[0],edge[1]] += 1
    return A

In [3]:
A = get_network(N=15000,p=0.005)

In [4]:
R = csr_matrix(A)

In [5]:
%t R.sum(axis=0)

1.7 ms ± 25.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%t R.sum(axis=1)

894 µs ± 58.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
from scipy.sparse import coo_matrix,spdiags,csr_matrix
from scipy.sparse.csgraph import laplacian
def version_1(A):
    N = A.shape[0]
    k_in = A.sum(axis=0)
    k_out = A.sum(axis=1).transpose()
    # form the graph laplacian
    operator = csr_matrix(
        spdiags(k_out+k_in,0,N,N)-A-A.transpose()
        )
    return operator,k_in,k_out
def version_2(A):
    N = A.shape[0]
    k_in = A.transpose().sum(axis=1)
    k_out = A.sum(axis=1)
    # form the graph laplacian
    operator = csr_matrix(
        spdiags((k_out+k_in).transpose(),0,N,N)-A-A.transpose()
        )
    return operator
def version_3(A):
    N = A.shape[0]
    T = A.transpose()
    k_in = T.sum(axis=1)
    k_out = A.sum(axis=1)
    # form the graph laplacian
    operator = csr_matrix(
        spdiags((k_out+k_in).transpose(),0,N,N)-A-T
        )
    return operator
def version_4(A):
    N = A.shape[0]
    T = A.transpose()
    k_in = T.sum(axis=0)
    k_out = A.sum(axis=1).transpose()
    # form the graph laplacian
    operator = laplacian(A+T)
    return operator

In [8]:
Op = []
Op.append(version_1(R))
Op.append(version_2(R))
Op.append(version_3(R))
Op.append(version_4(R))

In [9]:
table = np.zeros([4,4])
for idx,x in enumerate(Op):
    for idy,y in enumerate(Op):
        table[idx,idy] = (x!=y).nnz==0

In [10]:
table

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [11]:
%t version_1(R)

40.6 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%t version_2(R)

43.6 ms ± 5.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%t version_3(R)

40.6 ms ± 3.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%t version_4(R)

79.7 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
op,k_in,k_out = version_1(R)

In [41]:
def pad_1(k_in,k_out):
    solution_vector = np.append((k_out-k_in),np.matrix(0),axis=1).transpose()
    return solution_vector
def pad_2(k_in,k_out):
    solution_vector = np.zeros(len(k_out)+1)
    solution_vector[:len(k_out)] = (k_out-k_in).ravel().transpose()
    return solution_vector